# import das bibliotecas

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import rcParams
import numpy as np
from scipy import signal
from glob import glob
from scipy import stats

rcParams['figure.figsize'] = [16., 5.]

### Filtros

In [2]:
# definições de filtros

def butter_bandpass(data, lowcut, highcut, fs=200, order=4):
    nyq = fs * 0.5
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='bandpass')
    return signal.filtfilt(b, a, data)

def butter_lowpass(data, lowcut, fs=200, order=4):
    nyq = fs * 0.5
    low = lowcut / nyq
    b, a = signal.butter(order, low, btype='lowpass')
    return signal.filtfilt(b, a, data)

def butter_highpass(data, highcut, fs=200, order=4):
    nyq = fs * 0.5
    high = highcut / nyq
    b, a = signal.butter(order, high, btype='highpass')
    return signal.filtfilt(b, a, data)

def butter_notch(data, cutoff, var=1, fs=200, order=4):
    nyq = fs * 0.5
    low = (cutoff - var) / nyq
    high = (cutoff + var) / nyq
    b, a = signal.iirfilter(order, [low, high], btype='bandstop', ftype="butter")
    return signal.filtfilt(b, a, data)

### Carregando Dados

In [ ]:
# def plotData(dirs):
#     data = np.load("datasets/topicos_cc/"+dirs)
#     data = np.transpose(data, (0, 2, 1))
#     print(data.shape)
#     data_filtered = butter_notch(data, 60)
#     data_filtered = butter_highpass(data_filtered, 5)
#     data_filtered = butter_lowpass(data_filtered, 50)
#     for i in range(data_filtered.shape[1]):
#         plt.plot(data_filtered[0,i,:])
#     plt.suptitle(dirs)
#     plt.show()


In [3]:
def loadSujeito(dirs):
    data = np.load("datasets/topicos_cc/"+dirs)
    data = np.transpose(data, (0, 2, 1))
    data_filtered = butter_notch(data, 60)
    data_filtered = butter_highpass(data_filtered, 5)
    data_filtered = butter_lowpass(data_filtered, 50)
    
    return data_filtered

In [4]:
dirs = [ i.split("/")[-1] for i in glob('datasets/topicos_cc/p1*')]
data = []
for d in dirs:
    data.append(loadSujeito(d))

In [5]:
data_array = np.array(data)

In [6]:
data_array.shape

(3, 8, 4, 1600)

### Visualizando

In [ ]:
# for d in dirs:
#     plotData(d)

# Concatenando Trials x Movimentos

In [7]:
X_entrada = data_array[0]

X = []
X.append(data_array[1])
X.append(data_array[2])
X = np.array(X)

In [8]:
import numpy as np

#X = data.reshape(24, 4, 1600)
X = np.concatenate((np.array(X)), axis=0)

X.shape, X_entrada.shape

((16, 4, 1600), (8, 4, 1600))

## Segmentação dos dados

In [9]:
from scipy.signal import stft

def segmentacao(X):
    data = X
    step = 11.8
    segment = 128
    # data = X.reshape(24, 4, 1600)
    # print('', data.shape)

    n_win = int((data.shape[-1] - segment) / step) + 1
    ids = np.arange(n_win) * int(step)

    # Janelas do dado no dominio do tempo
    chunks_time = np.array([data[:,:,k:(k + segment)] for k in ids]).transpose(1, 2, 0, 3)

    # Janelas do dado no domínio da frequência
    _, _, chunks_freq = stft(data, fs=200, nperseg=128, noverlap=115)
    chunks_freq = np.swapaxes(chunks_freq, 2, 3)

    print('Formato (shape) dos dados depois da divisão de janelas')
    print(f'Dominio do tempo: {chunks_time.shape} - (classes+ensaios, canais, janelas, linhas)')
    print(f'Dominio da frequência:  {chunks_freq.shape} - (classes+ensaios, canais, janelas, linhas)')
    return chunks_time, chunks_freq


In [18]:
chunks_time, chunks_freq = segmentacao(X)
print()
chunks_time_entrada, chunks_freq_entrada = segmentacao(X_entrada)

Formato (shape) dos dados depois da divisão de janelas
Dominio do tempo: (16, 4, 125, 128) - (classes+ensaios, canais, janelas, linhas)
Dominio da frequência:  (16, 4, 125, 65) - (classes+ensaios, canais, janelas, linhas)

Formato (shape) dos dados depois da divisão de janelas
Dominio do tempo: (8, 4, 125, 128) - (classes+ensaios, canais, janelas, linhas)
Dominio da frequência:  (8, 4, 125, 65) - (classes+ensaios, canais, janelas, linhas)


In [19]:
chunks_time.shape, chunks_freq.shape

((16, 4, 125, 128), (16, 4, 125, 65))

## Achar as Janelas
O mesmo que a função acima, mas generico

In [ ]:
# from scipy.signal import stft

# def janela(overl=64):
    
#     for i in np.arange(1, 128, 0.1):
#         step = i
#         segment = 128
#         data = X.reshape(24, 4, 1600)
        
#         _, _, chunks_freq = stft(data, fs=200, nperseg=128, noverlap=overl)
#         chunks_freq = np.swapaxes(chunks_freq, 2, 3)
#         window = chunks_freq.shape[2] 

#         n_win = int((data.shape[-1] - segment) / step) + 1
#         ids = np.arange(n_win) * int(step)

#         # Janelas do dado no dominio do tempo
#         chunks_time = np.array([data[:,:,k:(k + segment)] for k in ids]).transpose(1, 2, 0, 3)
#         time_window = chunks_time.shape[2]
            
#         if( time_window == window ):
#             return step
        
    
# step = janela(overl=64)
# print(step)

In [ ]:
# all_steps = []
# for i in [0.5, 0.7, 0.8, 0.9 ]:
#     n_step = int(128*i)
#     print(n_step)
#     step = janela(overl=n_step)
#     all_steps.append(step)
    
# all_steps

In [ ]:
# from scipy.signal import stft

# def time_freq(step):
    
#     segment = 128
#     data = X.reshape(24, 4, 1600)
#     print('', data.shape)

#     n_win = int((data.shape[-1] - segment) / step) + 1
#     ids = np.arange(n_win) * int(step)

#     # Janelas do dado no dominio do tempo
#     chunks_time = np.array([data[:,:,k:(k + segment)] for k in ids]).transpose(1, 2, 0, 3)

#     # Janelas do dado no domínio da frequência
#     _, _, chunks_freq = stft(data, fs=200, nperseg=128, noverlap=64)
#     chunks_freq = np.swapaxes(chunks_freq, 2, 3)

#     print('Formato (shape) dos dados depois da divisão de janelas')
#     print(f'Dominio do tempo: {chunks_time.shape} - (classes+ensaios, canais, janelas, linhas)')
#     print(f'Dominio da frequência:  {chunks_freq.shape} - (classes+ensaios, canais, janelas, linhas)')
#     return chunks_time, chunks_freq

In [ ]:
# chunks_time , chunks_freq = time_freq(11.8)

## Extração e seleção de características

In [12]:
def getZC(data, th):
    tamanho = len(data)
    somatoria = 0
    
    for i in range(tamanho-1):
        resultado = (data[i] * data[i+1] )
        resultado2 = np.abs(data[i] - data[i+1])
        if(resultado < 0 ) and (resultado2 > th):
            somatoria += 1
        
    return somatoria

def fj(i, sampleRate, tamanho):
    return i * sampleRate / (2 * tamanho)

def getFMN(data):
    tamanho = len(data)
    somatoria = 0
    sumPSD = np.sum(PSD(data))
    for i in range(tamanho):
        somatoria += (fj(i, 41, tamanho) * PSD(data[i]) ) / sumPSD
        
    return somatoria



In [13]:
from math import prod

# funções auxiliares
def PSD(w):
    ''' definição da função PSD para o sinal no domínio da frequência '''
    return np.abs(w) ** 2

def wamp(x, th):
    res = np.abs(np.diff(x))
    return np.sum(res >= th, axis=-1)

def wl(x):
    res = np.abs(np.diff(x))
    return np.sum(res, axis=-1)

def var(x):
    return np.sum(x ** 2, axis=-1) / (np.prod(x.shape[:-1]) - 1)

def rms(x):
    return np.sqrt(np.sum(np.abs(x) ** 2, axis=-1) / (np.prod(x.shape[:-1])))

def fmd(w):
    return np.sum(PSD(w), axis=-1) / 2

def mmdf(w):
    return np.sum(np.abs(w), axis=-1) / 2

def zc(data,threshold):
    f =[]
    x,y,z = data.shape[:3]
    for xx in range(x):
        fx = []
        for yy in range(y):
            fy = []
            for zz in range(z):
                fy.append( getZC(data[xx][yy][zz], threshold ) )
            fx.append(fy)
        f.append(fx)
    return np.array(f)

def fmn(data):
    f =[]
    x,y,z = data.shape[:3]
    for xx in range(x):
        fx = []
        for yy in range(y):
            fy = []
            for zz in range(z):
                
                fy.append( getFMN(data[xx][yy][zz]) )
                
            fx.append(fy)
        f.append(fx)
    return np.array(f)

def A(w):
    return np.abs(w)

def getMMNF(data):
    tamanho = len(data)
    somatoria = 0
    
    sumA = np.sum(A(data))
    
    for i in range(tamanho):
        somatoria += (fj(i, 200, tamanho) * A(data[i]) ) / sumA 
        
    return somatoria

def mmnf(data):
    f =[]
    x,y,z = data.shape[:3]
    for xx in range(x):
        fx = []
        for yy in range(y):
            fy = []
            for zz in range(z):
                
                fy.append( getMMNF(data[xx][yy][zz]) )
                
            fx.append(fy)
        f.append(fx)
    return np.array(f)

def logD(data):
    from math import e
    N = np.prod(data.shape)
    
    return e ** ( np.sum(np.log10( np.abs(data) ), axis=-1) ) / N

def iemg(data):
    # tempo
    return np.sum(A(data), axis=-1)

def dasdv(data):
    #tempo
    return np.sqrt( np.sum(np.diff(data) ** 2, axis=-1) / (np.prod(data.shape[:-1]) - 1) )

def tmx(x, n):
    N = np.prod(x.shape[:-1])
    return np.abs(np.sum(x ** n, axis=-1) / N)

## Implementação do vetor

In [26]:
def final_filtros(chunks_time, chunks_freq, data):
    th = np.median(chunks_time)
    #VAR, RMS, WL, TM5 e DASDV
    #WL, IEMG, LOGD
    final_data = list()
    final_data.append(var(chunks_time))
    final_data.append(rms(chunks_time))
    final_data.append(wamp(chunks_time, th))
    final_data.append(logD(chunks_time))
    final_data.append(wl(chunks_time))
    final_data.append(zc(chunks_time,0))

    final_data.append(iemg(chunks_time))
    final_data.append(dasdv(chunks_time))
    final_data.append(tmx(chunks_time, 3))
    final_data.append(tmx(chunks_time, 4))
    final_data.append(tmx(chunks_time, 5))

    final_data.append(fmd(chunks_freq))
    final_data.append(mmdf(chunks_freq))
    final_data.append(fmn(chunks_freq))
    final_data.append(mmnf(chunks_freq))

    f, Pxx_den = signal.welch(data, fs=200, nperseg=248, noverlap=223)
    final_data.append(Pxx_den)
    final = np.array(final_data)
    print("SHAPE-: ",final.shape)

    return final
  

In [24]:


th = np.median(chunks_time)


final_data = list()
final_data.append(var(chunks_time))
final_data.append(rms(chunks_time))
final_data.append(wamp(chunks_time, th))
final_data.append(logD(chunks_time))
final_data.append(wl(chunks_time))
final_data.append(zc(chunks_time,0))

final_data.append(iemg(chunks_time))
final_data.append(dasdv(chunks_time))
final_data.append(tmx(chunks_time, 3))
final_data.append(tmx(chunks_time, 4))
final_data.append(tmx(chunks_time, 5))

final_data.append(fmd(chunks_freq))
final_data.append(mmdf(chunks_freq))
final_data.append(fmn(chunks_freq))
final_data.append(mmnf(chunks_freq))
# f, Pxx_den = signal.welch(data, fs=200, nperseg=248, noverlap=223)
# final_data.append(Pxx_den)

final = np.array(final_data)
print("SHAPE-: ",final.shape)



SHAPE-:  (15, 16, 4, 125)


In [27]:
final = final_filtros(chunks_time, chunks_freq, X)


SHAPE-:  (16, 16, 4, 125)


In [28]:
final_entrada = final_filtros(chunks_time_entrada, chunks_freq_entrada, X_entrada)

final.shape, final_entrada.shape

SHAPE-:  (16, 8, 4, 125)


((16, 16, 4, 125), (16, 8, 4, 125))

In [ ]:
# data = final.transpose(0, 1, 3, 2)
# sh = data.shape

# X = data.reshape(sh[0], int(sh[1]/3), 3 * sh[2], sh[3])

# print(X.shape)

## PCA


In [30]:
from sklearn.decomposition import PCA

In [ ]:
# pca = PCA(n_components=2)

# features = list()
# for f in X:
#     classes = list()
#     for c in f:
#         C_pca = pca.fit_transform(c)
#         classes.append(C_pca)
#     features.append(classes)

# X_pca = np.array(features)

In [ ]:
# X_pca.shape

## Visualização

In [ ]:
# def plot_features(features, features_names, classes_names, ch_1, ch_2):
    
#     movs = np.arange(len(classes_names))
#     markers = ["o", "v", "^", "P", "*", "x", "X", "2", "3", "1", 'm', 'L', 'z', 'U', '6']
#     for f, feature in enumerate(features):
        
#         for mov, marker in zip(movs, markers):
#             # argumentos: classes, amostras, canal
#             plt.scatter(feature[mov, :, ch_1],
#                         feature[mov, :, ch_2], marker=marker)

#         plt.legend((classes_names), scatterpoints=1, loc='best',
#                    ncol=3, fontsize=8)
        
#         plt.title(features_names[f])
#         plt.xlabel('CH{}'.format(ch_1))
#         plt.ylabel('CH{}'.format(ch_2))
#         plt.show()


In [ ]:


# import matplotlib.pyplot as plt
# from matplotlib import rcParams

# plt.rcParams["figure.figsize"] = (12, 12)

# features_name = ('var', 'rms', 'wamp', 'wl', 'zc','logd', 'iemg','dasdv','tm3','tm4','tm5', 'fmd', 'mmdf', 'fmn', 'mmnf')
# classes = [str(item) for item in list(range(8))]
# plot_features(X_pca, features_name, classes, 0, 1)


## Transpose para Selecionar Feature

In [31]:
final.shape
# 24*26 ,9, 4
# 24*26 , 10, 4
# 24*26 , 15, 4

(16, 16, 4, 125)

In [32]:
final.shape

(16, 16, 4, 125)

In [33]:
data = final.transpose(1, 3, 2, 0)
X = data.reshape(data.shape[0]*data.shape[1], data.shape[2]*data.shape[3])
X.shape

(2000, 64)

In [34]:
# y = np.array(list(range(1, 9)) * int(X.shape[0] / 8)) # Antigo

y = [ [(i)] * int(X.shape[0] / 8 ) for i in range(8)]
y = np.array(y).flatten()
y.shape

(2000,)

## Seleção de características

## Variance Threshold


In [ ]:
# # teste

# data_t = final.transpose(1, 3, 2, 0)
# ## X_t = data.reshape(24*26, 9, 4)
# ## X_t = data.reshape(24*26, 10, 4)
# ## X_t = data.reshape(24*26, 15, 4)
# X_t = data.reshape(24*26, 5, 4)

# data_t = X_t.transpose(2, 0, 1)
# data_t.shape


In [ ]:
# from sklearn.feature_selection import VarianceThreshold
# canais = list()

# for c in data_t:
#     sel = VarianceThreshold(threshold=(.1))
#     vt = sel.fit_transform(c)
#     canais.append(vt)


### RFE (Por causa do Kernel Linear não iremos utilizar)

In [ ]:
# from sklearn.feature_selection import RFE
# from sklearn.svm import SVC
# estimator = SVC(kernel="linear")
# selector = RFE(estimator, n_features_to_select=5, step=1)
# selector = selector.fit(X, y)
# s = selector.fit_transform(X, y)

### GenericUnivariateSelect

In [ ]:
# X.shape, y.shape

In [ ]:
# from sklearn.feature_selection import GenericUnivariateSelect, chi2
# transformer = GenericUnivariateSelect(chi2, mode='k_best', param=10)
# X_new = transformer.fit_transform(X, y)
# X_new.shape

In [ ]:
# from sklearn.feature_selection import SelectKBest
# X_new = SelectKBest(k=10).fit_transform(X, y)

In [ ]:
# X.shape, X_new.shape

## Normalização

In [35]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_norm = scaler.fit_transform(X)

# X_new_norm = scaler.fit_transform(X_new)

## SVM

In [36]:
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn import metrics

def do_svm(X,y):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, shuffle=True)

    clf = SVC(gamma='scale')
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    
    acc = metrics.accuracy_score(y_test, y_pred)
    
    return acc

do_svm(X,y), do_svm(X_norm,y)

(0.12166666666666667, 0.7166666666666667)

# Combinations

In [ ]:
# from itertools import combinations

# best_acc = 0
# best_comb = 0
# for comb in range(1,12):
#     for res in combinations(range(9),comb):
#         acc = do_svm(X_norm.take(res, axis=1), y)

#         if acc > best_acc:
#             best_acc = acc
#             best_comb = res

# print(f"Melhor Acurácia: {best_acc}, Melhor Combinação: {best_comb}")

# features_name = ('var', 'rms', 'wamp', 'wl', 'zc','logd', 'fmd', 'mmdf', 'fmn', 'mmnf')
# for i in best_comb:
#     print(f" {i} --- {features_name[i]}")


In [ ]:
# from itertools import combinations

# best_acc = 0
# best_comb = 0
# for comb in range(1,12):
#     for res in combinations(range(9),comb):
#         acc = do_svm(X.take(res, axis=1), y)

#         if acc > best_acc:
#             best_acc = acc
#             best_comb = res

# print(f"Melhor Acurácia: {best_acc}, Melhor Combinação: {best_comb}")

# features_name = ('var', 'rms', 'wamp', 'wl', 'zc','logd', 'fmd', 'mmdf', 'fmn', 'mmnf')
# for i in best_comb:
#     print(f" {i} --- {features_name[i]}")


In [ ]:
# from itertools import combinations

# best_acc = 0
# best_comb = 0
# best_k = 0
# for comb in range(1,12):
#     for res in combinations(range(9),comb):
#         for ks in range(1,41):
#             X_new = SelectKBest(k=ks).fit_transform(X, y)
#             acc = do_svm(X_new_norm.take(res, axis=1), y)

#             if acc > best_acc:
#                 best_acc = acc
#                 best_comb = res
#                 best_k = ks

# print(f"Melhor Acurácia: {best_acc}, Melhor Combinação: {best_comb}, Melhor K: {best_k}")




## RNN

In [37]:
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD, Adam
from keras.utils.np_utils import to_categorical
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import LabelEncoder
from urllib.request import urlopen, urlretrieve

import matplotlib.pyplot as plt
%matplotlib inline

2022-06-21 14:58:01.347896: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-21 14:58:01.347944: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Divisão dos dados em treino e teste:

In [45]:

# Dividindo em conjuntos de treino (80%) e teste (20%)
X_train, X_test, y_train, y_test = train_test_split(
    X_norm, y, test_size=0.3)

# treino: 80% dos 80% de treino. Validacao: 20% dos 80% de treino.
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.3, shuffle=None, stratify=y_train)


## Aplicação do algoritmo de MLP e geração dos resultados:

In [39]:
# definição de uma fração do regularizador
l = 0.01

# desenvolvimento do modelo Keras para uma MLP
model = Sequential()
model.add(Dense(20, activation='relu', input_dim=64,
                kernel_regularizer=regularizers.l2(l)))
# Aplicação de um dropout (caso necessário)
# model.add(Dropout(0.5))
model.add(Dense(10, activation='relu',
                kernel_regularizer=regularizers.l2(l)))
# Aplicação de um dropout (caso necessário)
# model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))

# Aplicação de um modelo de descida de gradiente utilizando o Stocastic Gradient Descendent (SGD)
sgd = SGD(lr=0.05, momentum=0.0)
# Função de otimização da rede: ADAM
adam = Adam(lr=0.005, beta_1=0.9, beta_2=0.999)
# Função de custo baseada em dados originalmente categóricos
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam,
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=550, batch_size=15,
                    validation_data=(X_val, y_val))

#Dro_out, batch_size, epoch, sgd, adam e l2

2022-06-21 14:58:05.063576: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-21 14:58:05.063627: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-21 14:58:05.063660: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (debian): /proc/driver/nvidia/version does not exist
2022-06-21 14:58:05.064916: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/alunos/a1858351/Documentos/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserW

Epoch 1/550
66/66 [==============================] - 1s 8ms/step - loss: 2.1966 - accuracy: 0.2439 - val_loss: 1.9828 - val_accuracy: 0.3167
Epoch 2/550
66/66 [==============================] - 0s 4ms/step - loss: 1.7618 - accuracy: 0.4071 - val_loss: 1.6829 - val_accuracy: 0.4095
Epoch 3/550
66/66 [==============================] - 0s 4ms/step - loss: 1.5546 - accuracy: 0.4847 - val_loss: 1.5068 - val_accuracy: 0.5405
Epoch 4/550
66/66 [==============================] - 0s 4ms/step - loss: 1.4394 - accuracy: 0.5633 - val_loss: 1.4502 - val_accuracy: 0.5429
Epoch 5/550
66/66 [==============================] - 0s 5ms/step - loss: 1.3654 - accuracy: 0.5827 - val_loss: 1.3942 - val_accuracy: 0.5976
Epoch 6/550
66/66 [==============================] - 0s 4ms/step - loss: 1.3020 - accuracy: 0.6337 - val_loss: 1.3434 - val_accuracy: 0.6381
Epoch 7/550
66/66 [==============================] - 0s 4ms/step - loss: 1.2670 - accuracy: 0.6418 - val_loss: 1.3660 - val_accuracy: 0.6000
Epoch 8/550
6

Epoch 59/550
66/66 [==============================] - 0s 4ms/step - loss: 0.6224 - accuracy: 0.9306 - val_loss: 0.8342 - val_accuracy: 0.8357
Epoch 60/550
66/66 [==============================] - 0s 4ms/step - loss: 0.6486 - accuracy: 0.9163 - val_loss: 0.8605 - val_accuracy: 0.8310
Epoch 61/550
66/66 [==============================] - 0s 4ms/step - loss: 0.6816 - accuracy: 0.9061 - val_loss: 0.8444 - val_accuracy: 0.8048
Epoch 62/550
66/66 [==============================] - 0s 4ms/step - loss: 0.6208 - accuracy: 0.9214 - val_loss: 0.8312 - val_accuracy: 0.8286
Epoch 63/550
66/66 [==============================] - 0s 4ms/step - loss: 0.6360 - accuracy: 0.9347 - val_loss: 0.8256 - val_accuracy: 0.8095
Epoch 64/550
66/66 [==============================] - 0s 4ms/step - loss: 0.6929 - accuracy: 0.8959 - val_loss: 0.9450 - val_accuracy: 0.8000
Epoch 65/550
66/66 [==============================] - 0s 4ms/step - loss: 0.6380 - accuracy: 0.9010 - val_loss: 0.8852 - val_accuracy: 0.8071
Epoch 

66/66 [==============================] - 0s 4ms/step - loss: 0.5713 - accuracy: 0.9204 - val_loss: 0.7458 - val_accuracy: 0.8524
Epoch 117/550
66/66 [==============================] - 0s 4ms/step - loss: 0.5365 - accuracy: 0.9357 - val_loss: 0.7652 - val_accuracy: 0.8405
Epoch 118/550
66/66 [==============================] - 0s 4ms/step - loss: 0.5212 - accuracy: 0.9449 - val_loss: 0.7461 - val_accuracy: 0.8476
Epoch 119/550
66/66 [==============================] - 0s 4ms/step - loss: 0.5118 - accuracy: 0.9418 - val_loss: 0.7039 - val_accuracy: 0.8595
Epoch 120/550
66/66 [==============================] - 0s 4ms/step - loss: 0.6592 - accuracy: 0.8765 - val_loss: 0.8995 - val_accuracy: 0.8190
Epoch 121/550
66/66 [==============================] - 0s 4ms/step - loss: 0.5979 - accuracy: 0.9092 - val_loss: 0.7653 - val_accuracy: 0.8310
Epoch 122/550
66/66 [==============================] - 0s 4ms/step - loss: 0.5983 - accuracy: 0.8990 - val_loss: 0.8584 - val_accuracy: 0.8143
Epoch 123/550

66/66 [==============================] - 0s 4ms/step - loss: 0.5075 - accuracy: 0.9418 - val_loss: 0.8600 - val_accuracy: 0.8119
Epoch 174/550
66/66 [==============================] - 0s 4ms/step - loss: 0.5346 - accuracy: 0.9276 - val_loss: 0.7536 - val_accuracy: 0.8357
Epoch 175/550
66/66 [==============================] - 0s 4ms/step - loss: 0.4747 - accuracy: 0.9429 - val_loss: 0.8195 - val_accuracy: 0.8405
Epoch 176/550
66/66 [==============================] - 0s 4ms/step - loss: 0.5026 - accuracy: 0.9429 - val_loss: 0.7490 - val_accuracy: 0.8357
Epoch 177/550
66/66 [==============================] - 0s 4ms/step - loss: 0.5328 - accuracy: 0.9245 - val_loss: 0.7841 - val_accuracy: 0.8238
Epoch 178/550
66/66 [==============================] - 0s 4ms/step - loss: 0.4831 - accuracy: 0.9449 - val_loss: 0.7511 - val_accuracy: 0.8333
Epoch 179/550
66/66 [==============================] - 0s 4ms/step - loss: 0.4775 - accuracy: 0.9449 - val_loss: 0.7489 - val_accuracy: 0.8429
Epoch 180/550

...

66/66 [==============================] - 0s 4ms/step - loss: 0.5225 - accuracy: 0.9071 - val_loss: 0.7543 - val_accuracy: 0.8357
Epoch 288/550
66/66 [==============================] - 0s 4ms/step - loss: 0.5031 - accuracy: 0.9337 - val_loss: 0.8286 - val_accuracy: 0.8095
Epoch 289/550
66/66 [==============================] - 0s 4ms/step - loss: 0.5569 - accuracy: 0.8929 - val_loss: 0.8130 - val_accuracy: 0.8286
Epoch 290/550
66/66 [==============================] - 0s 4ms/step - loss: 0.4786 - accuracy: 0.9378 - val_loss: 0.7525 - val_accuracy: 0.8524
Epoch 291/550
66/66 [==============================] - 0s 4ms/step - loss: 0.4734 - accuracy: 0.9429 - val_loss: 0.7088 - val_accuracy: 0.8738
Epoch 292/550
66/66 [==============================] - 0s 4ms/step - loss: 0.4110 - accuracy: 0.9643 - val_loss: 0.6266 - val_accuracy: 0.8952
Epoch 293/550
66/66 [==============================] - 0s 4ms/step - loss: 0.3835 - accuracy: 0.9724 - val_loss: 0.6578 - val_accuracy: 0.8762
Epoch 294/550

66/66 [==============================] - 0s 4ms/step - loss: 0.4729 - accuracy: 0.9316 - val_loss: 0.6736 - val_accuracy: 0.8524
Epoch 345/550
66/66 [==============================] - 0s 4ms/step - loss: 0.4953 - accuracy: 0.9265 - val_loss: 0.7543 - val_accuracy: 0.8452
Epoch 346/550
66/66 [==============================] - 0s 4ms/step - loss: 0.4711 - accuracy: 0.9398 - val_loss: 0.6407 - val_accuracy: 0.8810
Epoch 347/550
66/66 [==============================] - 0s 4ms/step - loss: 0.3971 - accuracy: 0.9673 - val_loss: 0.5893 - val_accuracy: 0.9071
Epoch 348/550
66/66 [==============================] - 0s 3ms/step - loss: 0.4216 - accuracy: 0.9480 - val_loss: 0.8055 - val_accuracy: 0.8310
Epoch 349/550
66/66 [==============================] - 0s 4ms/step - loss: 0.4379 - accuracy: 0.9459 - val_loss: 0.6591 - val_accuracy: 0.8643
Epoch 350/550
66/66 [==============================] - 0s 4ms/step - loss: 0.5287 - accuracy: 0.9194 - val_loss: 0.6721 - val_accuracy: 0.8500
Epoch 351/550

66/66 [==============================] - 0s 4ms/step - loss: 0.4602 - accuracy: 0.9429 - val_loss: 0.9735 - val_accuracy: 0.8214
Epoch 402/550
66/66 [==============================] - 0s 4ms/step - loss: 0.5405 - accuracy: 0.9041 - val_loss: 0.7122 - val_accuracy: 0.8738
Epoch 403/550
66/66 [==============================] - 0s 4ms/step - loss: 0.4369 - accuracy: 0.9439 - val_loss: 0.9118 - val_accuracy: 0.8214
Epoch 404/550
66/66 [==============================] - 0s 4ms/step - loss: 0.5864 - accuracy: 0.8959 - val_loss: 0.7510 - val_accuracy: 0.8500
Epoch 405/550
66/66 [==============================] - 0s 4ms/step - loss: 0.4550 - accuracy: 0.9347 - val_loss: 0.7121 - val_accuracy: 0.8643
Epoch 406/550
66/66 [==============================] - 0s 4ms/step - loss: 0.4267 - accuracy: 0.9459 - val_loss: 0.7011 - val_accuracy: 0.8833
Epoch 407/550
66/66 [==============================] - 0s 4ms/step - loss: 0.4166 - accuracy: 0.9582 - val_loss: 0.6756 - val_accuracy: 0.8619
Epoch 408/550

66/66 [==============================] - 0s 4ms/step - loss: 0.4308 - accuracy: 0.9469 - val_loss: 0.6622 - val_accuracy: 0.8738
Epoch 459/550
66/66 [==============================] - 0s 4ms/step - loss: 0.3766 - accuracy: 0.9694 - val_loss: 0.6610 - val_accuracy: 0.8952
Epoch 460/550
66/66 [==============================] - 0s 4ms/step - loss: 0.4038 - accuracy: 0.9571 - val_loss: 0.6473 - val_accuracy: 0.8667
Epoch 461/550
66/66 [==============================] - 0s 4ms/step - loss: 0.4827 - accuracy: 0.9306 - val_loss: 0.6999 - val_accuracy: 0.8571
Epoch 462/550
66/66 [==============================] - 0s 4ms/step - loss: 0.4796 - accuracy: 0.9276 - val_loss: 0.6693 - val_accuracy: 0.8810
Epoch 463/550
66/66 [==============================] - 0s 4ms/step - loss: 0.4772 - accuracy: 0.9265 - val_loss: 0.7834 - val_accuracy: 0.8595
Epoch 464/550
66/66 [==============================] - 0s 4ms/step - loss: 0.4227 - accuracy: 0.9582 - val_loss: 0.6954 - val_accuracy: 0.8810
Epoch 465/550

66/66 [==============================] - 0s 4ms/step - loss: 0.3696 - accuracy: 0.9653 - val_loss: 0.6649 - val_accuracy: 0.8857
Epoch 516/550
66/66 [==============================] - 0s 4ms/step - loss: 0.3904 - accuracy: 0.9602 - val_loss: 0.7929 - val_accuracy: 0.8381
Epoch 517/550
66/66 [==============================] - 0s 4ms/step - loss: 0.4280 - accuracy: 0.9429 - val_loss: 0.6887 - val_accuracy: 0.8714
Epoch 518/550
66/66 [==============================] - 0s 4ms/step - loss: 0.4656 - accuracy: 0.9276 - val_loss: 0.6650 - val_accuracy: 0.8690
Epoch 519/550
66/66 [==============================] - 0s 4ms/step - loss: 0.5209 - accuracy: 0.9214 - val_loss: 0.8775 - val_accuracy: 0.8095
Epoch 520/550
66/66 [==============================] - 0s 4ms/step - loss: 0.7816 - accuracy: 0.8388 - val_loss: 0.8806 - val_accuracy: 0.8071
Epoch 521/550
66/66 [==============================] - 0s 4ms/step - loss: 0.5120 - accuracy: 0.9306 - val_loss: 0.6623 - val_accuracy: 0.8738
Epoch 522/550

In [40]:
# 
# score = model.predict_classes(X_test)
# y_true = [np.where(x == 1)[0][0] for x in y_test]

predict_x=model.predict(X_test) 
score=np.argmax(predict_x,axis=1)
y_true = y_test

print('Acurácia: %0.2f%%' % (accuracy_score(y_true, score) * 100))
print('Matriz de confusão:')
print(confusion_matrix(y_true, score))
print()
print(classification_report(y_true, score, digits=5))

19/19 [==============================] - 0s 2ms/step
Acurácia: 88.67%
Matriz de confusão:
[[67  3  0  1  7  0  0  0]
 [ 0 84  0  2  1  3  3  0]
 [ 0  0 57  8  0  1  0  0]
 [ 1  0  0 65  0  0  1  1]
 [ 2  0  1  1 64  1  1  0]
 [ 3  4  0  0  5 58  2  2]
 [ 1  0  0  4  0  0 64  0]
 [ 2  0  0  3  4  0  0 73]]

              precision    recall  f1-score   support

           0    0.88158   0.85897   0.87013        78
           1    0.92308   0.90323   0.91304        93
           2    0.98276   0.86364   0.91935        66
           3    0.77381   0.95588   0.85526        68
           4    0.79012   0.91429   0.84768        70
           5    0.92063   0.78378   0.84672        74
           6    0.90141   0.92754   0.91429        69
           7    0.96053   0.89024   0.92405        82

    accuracy                        0.88667       600
   macro avg    0.89174   0.88720   0.88632       600
weighted avg    0.89414   0.88667   0.88745       600



# Grafico

In [ ]:
def plot_history(h):
    loss_list = [s for s in h.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in h.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in h.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in h.history.keys() if 'acc' in s and 'val' in s]
    if len(loss_list) == 0:
        print('Custo não está presente no histórico')
        return
    epochs = range(1, len(history.history[loss_list[0]]) + 1)
    # Custo
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, h.history[l], 'b',
                 label='Custo [treinamento] (' + str(str(format(
                    h.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, h.history[l], 'g',
                 label='Custo [validação] (' + str(str(format(
                    h.history[l][-1],'.5f'))+')'))
    plt.title('Custo')
    plt.xlabel('Épocas')
    plt.ylabel('Custo')
    plt.legend()
    # Acurácia
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, h.history[l], 'b',
                 label='Acurácia [treinamento] (' + str(format(
                    h.history[l][-1],'.5f'))+')')
    for l in val_acc_list:
        plt.plot(epochs, h.history[l], 'g',
                 label='Acurácia [validação] (' + str(format(
                    h.history[l][-1],'.5f'))+')')
    plt.title('Acurácia')
    plt.xlabel('Épocas')
    plt.ylabel('Acurácia')
    plt.legend()
    plt.show()

In [ ]:
plot_history(history)